In [1]:
import datetime
import numpy as np
import cv2 as cv
import os

from matplotlib import pyplot as plt
from pathlib import Path

from breathecam import BreatheCam
from cmn import make_video_div, write_mp4_video, write_video_html
from motion import MotionAnalysis, TemporalContour
from view import View

In [2]:
def write_events_html(
        contours: list[TemporalContour],
        events: list[np.ndarray], 
        path: str, 
        videos_dir: str, *, 
        title: str = "", 
        prefix: str = "event"):
    
    video_divs = []
    videos_dir = videos_dir.strip()

    if videos_dir.endswith('/'):
        videos_dir = videos_dir[:-1]

    Path(os.path.dirname(path)).mkdir(parents=True, exist_ok=True)
    Path(videos_dir).mkdir(parents=True, exist_ok=True)
    
    for i, contour in enumerate(contours):
        video_name = f"{videos_dir}/{prefix}{i + 1}.mp4"

        try:
            write_mp4_video(video_name, events[i])
            size = events[i].shape[0:3]

            metadata = [
                f"Video {i + 1}",
                f"Start Frame: {contour.frames[0]}"
                f"Dimensions: {size[0]} {size[2]}x{size[1]} frames",
                f"Points in Contour: {contour.number_of_points}",
                f"Contour Area: {contour.width * contour.height}", 
                f"Point Density in Contour Area: {contour.density(5)}"
            ]

            video_divs.append(make_video_div(video_name, metadata))
        except BrokenPipeError as e:
            print(e)

    write_video_html(path, '\n'.join(video_divs), title)

In [3]:
day = datetime.date.fromisoformat("2024-05-19")
nlevels = 4
#view = View(4653, 2127, 4915, 2322)
view = View(2307, 1914, 6814, 2515)
view_ss = view.subsample(nlevels)
day_time = datetime.time.fromisoformat("09:49:00")
breathecam = BreatheCam.init_from("Clairton Coke Works", day)

In [7]:
def run_analysis(frames: int, background_subtractor):    
    print(f"Size: {view.width}x{view.height} ({view.width * view.height} pixels)")
    print(f"Subsampled Size: {view_ss.width}x{view_ss.height} ({view_ss.width * view_ss.height})")    

    return breathecam.analyze(day_time, view, frames, nlevels=nlevels, background_subtractor=background_subtractor)


def get_no_white_pixels(analysis: MotionAnalysis, neighbors: int = 8, depth: int = 3) -> list[TemporalContour]:
    print(f"Neighbors: {neighbors}")
    print(f"Temporal Window: {depth}")

    hls_video = np.array([cv.cvtColor(f, cv.COLOR_RGB2HLS) for f in analysis.video])

    contours = analysis.contours(neighbors=neighbors, depth=depth)
    contours = [c for c in contours if c.number_of_points > 100]
    contours = [c for c in contours if not analysis.has_white_pixel(c, hls_video, nlevels=1, lightness_lower_bound=160)]
    
    print(f"Events with no white pixels: {len(contours)}\n")
    
    contours.sort(key=lambda c: c.number_of_points, reverse=True)

    return contours

    
def emit_events(analysis: MotionAnalysis,
                contours: list[TemporalContour], 
                path: str, 
                videos_dir: str, *, 
                title: str = "", 
                prefix: str = "event"):

    full_res_video = breathecam.download_video(day_time, analysis.number_of_frames, view, nlevels=1)
    events = [analysis.get_contour(c, full_res_video, nlevels=nlevels, pad_frames=5, pad_region=128) for c in contours]
    write_events_html(contours, events, path, videos_dir, title = title, prefix=prefix)

In [5]:
nframes = 40
bgsub_type = "MOG2"
bgsub = cv.createBackgroundSubtractorMOG2(detectShadows=True, varThreshold=9)
analysis = run_analysis(nframes, bgsub)
contours_no_white = get_no_white_pixels(analysis, neighbors=8, depth=3)

Size: 4507x601 (2708707 pixels)
Subsampled Size: 1127x151 (170177)
Neighbors: 8
Temporal Window: 3
Events with no white pixels: 50



In [8]:
emit_events(
        analysis, 
        contours_no_white, 
        "./events_1800_frames_no_white.html",
        "smoke_detect/events/1800",
        title="Events - 1800 Frames - No White Pixels",
        prefix="event_")